### L’API OpenAI

#### Installation des dépendences

Pour la clé d'api OpenAI, il est indispensable de la sécuriser. Pour cela plusieurs possibilités selon le contexte:

* Via Google Colab: créez un secret Google Colab nommé `OPENAI_API_KEY` (menu de gauche) puis utiliser `google.colab.userdata.get` pour la récupérer

* En local: créez un fichier `.env`, déclarez une paire clé-valeur `OPENAI_API_KEY=votre_clé` puis utiliser `python-dotenv` pour la charger dans les variables d'environnement

Quoi qu'il arrive ne **JAMAIS** partager ou commiter vos clés d'api.

En dehors de la gestion de la clé, nous installons le wrapper de l'api OpenAI.

In [ ]:
%pip install openai -q
#%pip install python-dot-env -q

In [ ]:
"""
Clé depuis secrets Google Colab
Setter la clé dans les secrets dans le menu de gauche
"""
from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')

"""
Clé OPENAI_API_KEY depuis fichier .env local.
Uncomment les lignes suivantes
"""
#from dotenv import load_dotenv
#from os import getenv
#load_dotenv()
#api_key=getenv('OPENAI_API_KEY')

#### Création de la requête

* temperature - entre 0 et 1. Une valeur haute, comme 0.8 entraînera une complétion plus aléatoire. Alors qu'une valeur faible, comme 0.2 donnera une complétion plus déterminste
* messages - La liste des messages de la conversation
* max_completion_tokens - Permet de limiter le nombre de token de la complétion, pour pouvoir par exemple contrôler le coût de la génération.

> [API reference](https://platform.openai.com/docs/api-reference/chat/create)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explique moi la théorie de la relativité en termes simples",
        }
    ],
    model="gpt-4o-mini"
)



La réponse contient d'autres infos en plus de la réponse du LLM. Notamment les stats d'usage pour l'appel (nombre de token du prompt input et de la complétion).

In [ ]:
chat_completion

In [ ]:
chat_completion.usage

La complétion est, elle, accessible dans l'objet sous le chemin `choices[0].message.content`

In [ ]:
from IPython.display import display, Markdown
display(Markdown(chat_completion.choices[0].message.content))

## Cas pratique :

1. Créer des requêtes simples vers un modèle de langage (LLM) via l’API.
2. Analyser les résultats générés en fonction des prompts et comprendre comment le modèle traite l'information.
3. Expérimenter différents types de prompts pour explorer les capacités du modèle et voir comment les réponses varient.

### Exercice 1 : Prompts descriptifs
Envoyez un prompt pour obtenir une explication simple d'un concept technique.
prompt = "Explique-moi le fonctionnement d'un moteur à combustion interne."

### Exercice 2: Questions ouvertes vs fermées
Posez des questions ouvertes et des questions fermées pour voir comment le modèle réagit.
prompt = "Quels sont les défis actuels du changement climatique ?"
prompt = "Le changement climatique affecte-t-il la montée du niveau de la mer ?"

### Exercice 3: Génération créative
Essayez de jouer avec la température (ex. 0.3 vs 0.9) et observez les différences :
prompt = "En 3 phrases, rRaconte une courte histoire où un robot devient ami avec un humain."



